In [1]:
if True:
    from tqdm.notebook import tqdm
    import imageio.v2 as imageio
    from mpl_toolkits.axes_grid1 import ImageGrid
    import matplotlib.cm as cm
    import matplotlib.animation as animation
    import gc
    from glob import glob
    import warnings 
    warnings.filterwarnings("ignore")
    import os
    from random import sample, seed, randint, random, shuffle, choice, choices, uniform, gauss, triangular
    from PIL import Image, ImageOps 
    from pandas_datareader import data as pdr
    import pandas as pd
    import shutil
    from collections import OrderedDict
    os.chdir("/home/smoothjazzuser/videogame-anomoly/MNAD/")
    # get current dir
    os.getcwd()
    import numpy as np
    import sys
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    import torchvision
    import torch.nn.init as init
    import torch.utils.data as data
    import torch.utils.data.dataset as dataset
    import torchvision.datasets as dset
    import torchvision.transforms as transforms
    from torch.autograd import Variable
    import torchvision.utils as v_utils
    import matplotlib.pyplot as plt
    import cv2
    import math
    from collections import OrderedDict
    import copy
    import time
    from model.utils import DataLoader
    from model.final_future_prediction_with_memory_spatial_sumonly_weight_ranking_top1 import *
    from model.Reconstruction import *
    from sklearn.metrics import roc_auc_score
    import random
    import subprocess
    import argparse
    import matplotlib.pyplot as plt
    from time import sleep
    import matplotlib.image as mpimg
    import shutil
    import cv2
    from scipy.stats import percentileofscore
    import PIL
    from joblib import Parallel, delayed
    mse = nn.MSELoss(reduction='none')
    from torchvision.transforms.functional import gaussian_blur
    from model.utils import DataLoader
    from utils import *
    from model.Reconstruction import *
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

In [2]:
# all vars to leep track of. Load previous data if it exists, and extend it where new info is present
collumns = ['SCORE','FP_WINDOW', 'CHANGE_WINDOW', 'FP', 'TP', 'ANOMALIES', 'ARCHITECHURE', 'EPOCHS', 'INFERENCE_LOSS_TYPE', 'TRAINING_LOSS_TYPE', "KERNEL_SIZE"]
if os.path.exists('results.csv'):
    df = pd.read_csv('results.csv')
    df = df[collumns]
    df
else:
    df = pd.DataFrame(columns=collumns)
    df.to_csv('results.csv', index=False)
df.head()

,SCORE,FP_WINDOW,CHANGE_WINDOW,FP,TP,ANOMALIES,ARCHITECHURE,EPOCHS,INFERENCE_LOSS_TYPE,TRAINING_LOSS_TYPE,KERNEL_SIZE


In [3]:
#vars 
loss_sections = 1
temp_dir = "/home/smoothjazzuser/Desktop/ram/"
methodd = "median_blur"         #'fourier_blur' # False, 'fourier_blur', 'cummulative_blur', 'median_blur'
downscale = True
dims = 84
channels = 3
video = False
FP_WINDOW = 60
SDEV_SENSITIVITY = 1 # not connected to any vars yet. Scale with the std under the data dist curve... it means something mathematically
CHANGE_WINDOW = 10
KERNEL_SIZE = 5

if not os.path.exists(temp_dir + "temp/"):
    os.mkdir(temp_dir + "temp/")

if not os.path.exists(temp_dir + "cleaned/"):
    os.mkdir(temp_dir + "cleaned/")
temp_dir = temp_dir + "temp/"
std_loss_correction = {i:[] for i in range(loss_sections)} #False
histories = {}

for i in range(3):
    histories[f"d{i}"] = []
    histories[f"x{i}"] = []
    histories[f"y{i}"] = []
    histories[f"t{i}"] = []

In [4]:
#Functions
if True:

    def normalize_list(list:list):
            min_val = min(list)
            max_val = max(list)
            return [(x-min_val)/(max_val-min_val) for x in list]

    def FP_TP (labels, preds):
        PREDS = preds.index
        TP = 0
        FP = 0
        FN = 0
        TOTAL_ANOMALIES = {}#{'anomalous':[i for i in range(len(labels)) if labels[i] == 1]}
        #find the places where the labels break sequence
        
        # find total anomalies
        TOTAL_ANOMALIES = []
        for i in range(len(labels)):
            if i < CHANGE_WINDOW:
                continue
            num_groups = len(TOTAL_ANOMALIES)
            
            if labels[i] == 1 and labels[i-1] == 0:
                TOTAL_ANOMALIES.append([i])
                
            elif labels[i] == 1 and labels[i-1] == 1:
                if num_groups == 0:
                    TOTAL_ANOMALIES.append([i])
                else:
                    TOTAL_ANOMALIES[num_groups - 1].append(i)

        # find TP and FP
        anomally_assignments = {index:[] for index, i in enumerate(TOTAL_ANOMALIES)}
        irrelevent = []
        for pred_index in PREDS:
            if pred_index < CHANGE_WINDOW:
                continue

            n_within_window = [l for l in range(pred_index-FP_WINDOW, pred_index+FP_WINDOW + 1)]

            # if any value in n_within_window is also in TOTAL_ANOMALIES
            valid_pred = False

            for win in n_within_window:
                for bin_values in TOTAL_ANOMALIES:
                    if win in bin_values:
                        valid_pred = True
                        anomally_assignments[TOTAL_ANOMALIES.index(bin_values)].append(1)
                        break
            if not valid_pred:
                false_preds_window =  list(range(pred_index-FP_WINDOW, pred_index+FP_WINDOW + 1))
                if len(set(false_preds_window).intersection(irrelevent)) == 0:
                    FP += 1
                    irrelevent.extend(false_preds_window)
                else:
                    # merge the two lists
                    irrelevent = list(set(false_preds_window).union(irrelevent))
                    
        for key, value in anomally_assignments.items():
            if sum(value) > 0:
                TP += 1
            else:
                FN += 1

        return f"TP_{TP}_FP_{FP}_FN_{FN}_ANOMALIES_{len(TOTAL_ANOMALIES)}", (TP, FP, FN, len(TOTAL_ANOMALIES), TOTAL_ANOMALIES, anomally_assignments)

    def load_(filename):
        i = plt.imread(filename)
        return  i

    def load_frames(i, h, w): 
        if os.path.exists(f"/tmp/{i}.jpg"):
            return np.uint8(cv2.resize(imageio.imread(f"/tmp/{i}.jpg"), (h,w)))
        else:
            return np.uint8(cv2.resize(imageio.imread(f"/tmp/{i}.png"), (h,w)))

    def normalize_array(array:np.ndarray):
        min_val = np.min(array)
        max_val = np.max(array)
        return (array-min_val)/(max_val-min_val)

    def split_list(list:list, n:int):
        return [list[i:i+n] for i in range(0, len(list), n)]

    def plot_images(i, x, diffs, ground_truth, label_plot, preds, points):
            window = 50
            if i > 50: window = i - 50
            else: window = 0
            
            #plot preds
            fig = plt.figure()
            fig.set_size_inches(10, 10, forward=True)
            ax = fig.add_subplot(2,2,1)
            ax.set_title(f"prediction {window}:{i}")
            ax.imshow(preds, cmap='gray') 

            #plot diffs
            ax = fig.add_subplot(2,2,2)
            ax.set_title(f"diff {window}:{i}")
            ax.imshow(diffs, cmap='gray')

            #plot ground_truth
            ax = fig.add_subplot(2,2,4)
            ax.set_title(f"ground truth {window}:{i}")
            ax.imshow(ground_truth, cmap='gray')

            #plot anomaly score
            ax = fig.add_subplot(2,2,3)
            ax.set_title(f"anomaly score {window}:{i}")
            ax.plot(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', linewidth=0.6)
            ax.scatter(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', s=4)
            ax.scatter(np.arange(len(label_plot['x'][window:i])), label_plot['y'][window:i], color='red', marker='x', s=4)

            # find all search.points that are in the window
            matching = [(i,points[x]) for i, x in enumerate(np.arange(window, i)) if x in points.index]
            # plot these points with y-values = data y values
            ax.scatter([x[0] for x in matching], [x[1] for x in matching], label='Flagged points', color='purple', s=14, alpha=0.5, marker='o')


            #ax.legend([metric])
            fig.tight_layout()
            fig.subplots_adjust(wspace=0, hspace=0)
            #set ax to show 50 x values
            ax.set_xlim(0, 50)
            ax.set_ylim(0, 1)
            fig.savefig(f"/tmp/{i}.jpg", bbox_inches='tight', pad_inches=0)
            
            fig.clear()
            plt.close(fig)
            del ax
            del window
            del fig
            return None

    def percentile  (array, percentile):
        percentile = np.squeeze(percentile)
        array = np.squeeze(array)
        return np.percentile(array, percentile)

    def copy_frames(folder, destination, keep_every):
                files = sorted(glob(os.path.join(folder, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0])) + sorted(glob(os.path.join(folder, '*.png')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                dest = os.path.join(destination, folder.split('/')[-1])
                for i, file in enumerate(files):
                    if i % keep_every == 0:
                        shutil.copy(file, os.path.join(dest, os.path.basename(file)))
                # rename all files in the folder to be sequential, starting at 0
                files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0])) + sorted(glob(os.path.join(dest, '*.png')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                for i, file in enumerate(files):
                    os.rename(file, os.path.join(dest, str(i).zfill(3)+'.jpg'))

                # remove frames until len(files) is a multiple of 5
                files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0])) + sorted(glob(os.path.join(dest, '*.png')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                while len(files) % 1 != 0:
                    os.remove(files[-1])
                    files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                del files, dest, folder, destination, keep_every

    def bl (x, y, ix, id):
        x = cv2.GaussianBlur(x, (5,5), 0)
        y = cv2.GaussianBlur(y, (5,5), 0)

        x = np.squeeze(x)
        y = np.squeeze(y)
        
        x = np.fft.fft2(x)
        y = np.fft.fft2(y)
        x = np.fft.fftshift(x)
        y = np.fft.fftshift(y)
        #set all values below 0.1 to 0
        x_h = np.percentile(np.abs(x), 100)
        y_h = np.percentile(np.abs(y), 100)
        x_l = np.percentile(np.abs(x), 90)
        y_l = np.percentile(np.abs(y), 90)
        x = np.where((np.abs(x) > x_h) &  (np.abs(x) < x_l), 0, x)
        y = np.where((np.abs(y) > y_h) &  (np.abs(y) < y_l), 0, y)
        x = np.fft.ifftshift(x)
        y = np.fft.ifftshift(y)
        x = np.fft.ifft2(x)
        y = np.fft.ifft2(y)
        
        x = np.abs(x)
        y = np.abs(y)

        x = cv2.GaussianBlur(x, (5,5), 0)
        y = cv2.GaussianBlur(y, (5,5), 0)

        d = np.abs(x-y)
        for _ in range(10):
            d = cv2.GaussianBlur(d, (11,11), 0)

        histories[str(id)].append(d)
        if len(histories[str(id)]) > 25: 
            histories[str(id)].pop(0)
        d = np.abs(d - np.mean(np.array(histories[str(id)]) , axis=0)) + 1e-9
        # for _ in range(10):
        #     d = cv2.GaussianBlur(d, (11,11), 0)
        thresh_d = np.percentile(d, 90)
        histories['t'+str(id)[1]].append(thresh_d)
        if len(histories['t'+str(id)[1]]) > 10:
            histories['t'+str(id)[1]].pop(0)
        d = np.where(d < np.mean(histories['t'+str(id)[1]]), 0, d)

        return d

    def cb(x, y, ix, id):
        global histories
        
        x = np.squeeze(x)
        y = np.squeeze(y)

        d = np.abs(x-y) + 1e-9
        
        for i in range(4):
            d = cv2.GaussianBlur(d, (11,11), 0)

        if type(histories[str(id)]) == list:
            histories[str(id)] = d

        d = np.abs(d - histories[str(id)]) + 1e-9

        histories[str(id)] = (histories[str(id)] + d/10) / (1 + 1/10)
        #d = np.where(d < histories[str(id)], d, histories[str(id)])

        return d

    def mb(x, y, ix, id):
        x = np.squeeze(x)
        y = np.squeeze(y)
        x = x * 255
        y = y * 255
        x = x.astype(np.uint8)
        y = y.astype(np.uint8)
        for i in range(1):
            x = cv2.medianBlur(x, KERNEL_SIZE)
            y = cv2.medianBlur(y, KERNEL_SIZE)
        d = np.abs(np.subtract(x,y), dtype=np.float32)**2
        return d

    def gb(x, y, ix, id):
        x = np.squeeze(x)
        y = np.squeeze(y)
        x = x * 255
        y = y * 255
        x = x.astype(np.uint8)
        y = y.astype(np.uint8)
        for i in range(1):
            x = cv2.GaussianBlur(x, (KERNEL_SIZE,KERNEL_SIZE), 0)
            y = cv2.GaussianBlur(y, (KERNEL_SIZE,KERNEL_SIZE), 0)
        d = np.abs(np.subtract(x,y), dtype=np.float32)**2
        return d
    
    def mse(x, y, ix, id):
        x = np.squeeze(x)
        y = np.squeeze(y)
        d = np.abs(np.subtract(x,y), dtype=np.float32)**2
        return d

    def loss_func_mse(x, y, fft=True, colors=True, method=methodd, std_details=True, diff_ = True):
        global loss_sections, std_loss_correction, histories
        losses = {'x':[], 'y':[]}
        x.transpose_(1,3)
        y.transpose_(1,3)
        x = x.cpu().detach().numpy()
        y = y.cpu().detach().numpy()
        x = np.squeeze(x)
        y = np.squeeze(y)
        x = cv2.resize(x, (dims, dims))
        y = cv2.resize(y, (dims, dims))

        if methodd:
            d = np.zeros(x.shape)
            if methodd == 'fourier_blur':
                x = x.astype(np.float64)
                y = y.astype(np.float64)
                for i in range(x.shape[0]): # we are using color images
                    d[:,:,i] = bl(x[i,:,:], y[i,:,:], f'x{i}', f'd{i}')
            
            elif methodd == 'cummulative_blur':
                x = x.astype(np.float64)
                y = y.astype(np.float64)
                for i in range(x.shape[0]):
                    d[:,:,i] = cb(x[i,:,:], y[i,:,:], f'x{i}', f'd{i}')

            elif methodd == 'median_blur' or methodd == 'MSE_BLUR':
                for i in range(channels):
                    d[:,:,i] = mb(x[:,:,i], y[:,:,i], f'x{i}', f'd{i}')

            elif methodd == 'gaussian_blur' or methodd == 'G_BLUR':
                for i in range(channels):
                    d[:,:,i] = gb(x[:,:,i], y[:,:,i], f'x{i}', f'd{i}')

            else:
                for i in range(channels):
                    d[:,:,i] = mse(x[:,:,i], y[:,:,i], f'x{i}', f'd{i}')

            loss = d

        else:
            #loss = F.mse_loss(x, y)
            loss = np.abs(x-y)**2

        return loss#**2

In [5]:
class anomaly():
  
    def __init__(self, method, metric, std_correction, dataset):
        self.method, self.metric, self.std_correction, self.dataset = method, metric, std_correction, dataset
        self.model = False
        self.m_items = False
   
    def swap_test_folders(self, inference='01'):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.inference = inference
        #check if frames exists:
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
            !rm -r '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'

        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'
        else:
            if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
                !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/'$inference '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$inference
        ##############################################################################
        frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)
        # expand dims and save again
        #frame_labels_bugs = np.expand_dims(frame_labels_bugs, axis=0)
        frame_labels_bugs[0]
        #np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/frame_labels_bugs.npy', frame_labels_bugs)
        if True:
            frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)

            # list directories in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/ using glob
            # glob returns a list of all files in the directory
            dd = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/*')
            dd.sort()
            directories = [x.split('/')[-1] for x in dd if x.split('/')[-1].isdigit()]
            directories.sort()
            directories = {d:len(glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d + '/*.jpg') + glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d + '/*.png')) for d in directories if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d)}
            directories = OrderedDict(sorted(directories.items(), key=lambda x: int(x[0])))
            print(list(directories))
            iterate = 0
            for folder, num_frames in directories.items():
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + folder):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$folder

                    print(folder, num_frames)
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy'):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$folder
                    # seperate the labels for each folder from the frame_labels_bugs
                    print(folder, num_frames)
                    labels = frame_labels_bugs[0][iterate:iterate+int(num_frames)]
                    iterate += num_frames
                    np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy', np.array(np.expand_dims(np.array(labels), axis=0)))
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'):
            !rm '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        else:
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$inference'/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'

        test = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy', allow_pickle=True)
        print(test.shape)
        (sum(directories.values()))

    def Evaluate(self, c=channels, loss_sections=1, method='blur', method_ = 'recon', dataset_type='bugs', image_size=dims, metric_only=False):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/MNAD/")
        std_loss_correction = {i:[] for i in range(loss_sections)} #False

        args = {'gpus':'','batch_size':5,'test_batch_size':1,'h':image_size,'w':image_size,'c':c,'method':method_,'t_length':1 if method_=='recon' else 5,'fdim':512,'mdim':512, 'msize':10,'alpha':0.6,'th':0.0000,'num_workers':8,'num_workers_test':1,'dataset_type':'bugs','dataset_path':'./dataset','model_dir':f'./exp/{dataset_type}/{method_}/log/model.pth','m_items_dir':f'./exp/{dataset_type}/{method_}/log/keys.pt'}
        

        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
        gpus = "0"
        os.environ["CUDA_VISIBLE_DEVICES"]= gpus
        torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance
        test_folder = args['dataset_path']+"/"+args['dataset_type']+"/testing/frames"
        test_dataset = DataLoader(test_folder, transforms.Compose([transforms.ToTensor(),]), resize_height=args['h'], resize_width=args['w'], time_step=args['t_length']-1)
        test_size = len(test_dataset)
        print("The number of test data is %d" % test_size)
        test_batch = data.DataLoader(test_dataset, batch_size = args['test_batch_size'], shuffle=False, num_workers=args['num_workers_test'], drop_last=False)

        ###############################
        histories = {}
        for i in range(args['c']):
            histories[f"d{i}"] = []
            histories[f"x{i}"] = []
            histories[f"y{i}"] = []
            histories[f"t{i}"] = []

            
        #######################################################
        # Loading the trained model
        if not self.model: 
            model = torch.load(args['model_dir'])
            model.cuda()
            m_items = torch.load(args['m_items_dir'])
        else:
            model = self.model
            m_items = self.m_items
        labels = np.load('./data/frame_labels_'+args['dataset_type']+'.npy')

        videos = OrderedDict()

        videos_list = sorted(glob(os.path.join(test_folder, '*')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        for video in videos_list:
            video_name = video.split('/')[-1]
            videos[video_name] = {}
            videos[video_name]['path'] = video
            videos[video_name]['frame'] = sorted(glob(os.path.join(video, '*.jpg')) + glob(os.path.join(video, '*.png')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            #videos[video_name]['frame'].sort()
            videos[video_name]['length'] = len(videos[video_name]['frame'])

        labels_list = []
        label_length = 0
        psnr_list = {}
        feature_distance_list = {}

        print('Evaluation of', args['dataset_type'])

        # Setting for video anomaly detection
        for video in sorted(videos_list, key=lambda x: int(x.split('/')[-1].split('.')[0])):
        #for video in videos_list:
            video_name = video.split('/')[-1]
            if args['method'] == 'pred':
                labels_list = np.append(labels_list, labels[0][4+label_length:videos[video_name]['length']+label_length])
            else:
                labels_list = np.append(labels_list, labels[0][label_length:videos[video_name]['length']+label_length])
            label_length += videos[video_name]['length']
            psnr_list[video_name] = []
            feature_distance_list[video_name] = []

        label_length = 0
        video_num = 0
        label_length += videos[videos_list[video_num].split('/')[-1]]['length']
        m_items_test = m_items.clone()

        model.eval()
        kkk=0
        ccc = int(test_size/args['test_batch_size'])
        if not metric_only: diffs = []
        if not metric_only: preds = []
        if not metric_only: ground_truths = []
        label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{args['dataset_type']}.npy", allow_pickle=True).tolist()[0]
        loss_hist = {i:[] for i in range(loss_sections)}
        for k,(imgs) in enumerate(test_batch):
            
            if k == label_length:
                video_num += 1
                label_length += videos[videos_list[video_num].split('/')[-1]]['length']

            imgs = Variable(imgs).cuda()
            outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, compactness_loss = model.forward(imgs, m_items_test, False)
            mse_imgs = np.sum(loss_func_mse(outputs, imgs))#.item()
            mse_feas = compactness_loss.item()
            if not metric_only:
                diff = loss_func_mse(outputs, imgs)
                #diff = normalize_array(diff)
                diffs.append(diff)

                pred = (outputs[0].detach().cpu().numpy()+1)/2
                pred = pred.transpose(1,2,0)
                preds.append(pred)

            kkk+=1

            psnr_list[videos_list[video_num].split('/')[-1]].append(psnr(mse_imgs))
            feature_distance_list[videos_list[video_num].split('/')[-1]].append(mse_feas)
            print(f"pairs pred: {round(100*kkk/ccc,3)}%, mse_feas:{mse_feas}", end = "\r")


        # Measuring the abnormality score and the AUC
        anomaly_score_total_list = []
        for video in sorted(videos_list):
            video_name = video.split('/')[-1]
            anomaly_score_total_list += score_sum(anomaly_score_list(psnr_list[video_name]), 
                                            anomaly_score_list_inv(feature_distance_list[video_name]), args['alpha'])

        anomaly_score_total_list = np.asarray(anomaly_score_total_list)

        if not os.path.exists(f"{temp_dir}{args['dataset_type']}"):
            os.mkdir(f"{temp_dir}{args['dataset_type']}")
        if not os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}"):
            os.mkdir(f"{temp_dir}{args['dataset_type']}/{args['method']}")
        if not os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log"):
            os.mkdir(f"{temp_dir}{args['dataset_type']}/{args['method']}/log")

        if os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/"):
            l = f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/"
            shutil.rmtree(l)
        if os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/"):
            l = f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/"
            shutil.rmtree(l)
        if os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/ground_truths/"):
            l = f"{temp_dir}{args['dataset_type']}/{args['method']}/log/ground_truths/"
            shutil.rmtree(l)

        if not os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/"):
            os.makedirs(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/")
        if not os.path.exists(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/"):
            os.makedirs(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/")

        np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/anomaly_score_total_list.npy", anomaly_score_total_list)
        np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/anomaly_score_total_list.npy", anomaly_score_total_list)
        np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/psnr_list.npy", psnr_list)
        np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/feature_distance_list.npy", feature_distance_list)
        #np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/auc.npy", accuracy)
        np.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/labels_list.npy", labels_list)

        
        if not metric_only:
            for (i,img) in enumerate(preds): 
                #use pillow to save the image in grayscale
                if args['c'] == 1:
                    img = img*255
                    img = img.astype(np.uint8)
                    img = np.squeeze(img)
                    img = PIL.Image.fromarray(img, 'L')
                    img.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/{i}.jpg")
                else:
                    img = img*255
                    img = img.astype(np.uint8)
                    img = PIL.Image.fromarray(img, 'RGB')
                    img.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/preds/{i}.jpg")
            if methodd == False or methodd != False:
                for (i,img) in enumerate(diffs): 
                    if args['c'] == 1:
                        img = img*255
                        img = img.astype(np.uint8)
                        img = np.squeeze(img)
                        img = PIL.Image.fromarray(img, 'L')
                        img.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/{i}.jpg")
                    else:
                        img = img*255
                        img = img.astype(np.uint8)
                        img = PIL.Image.fromarray(img, 'RGB')
                        img.save(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/{i}.jpg")
            else:
                shutil.rmtree(f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/")
                shutil.copytree(f"{temp_dir}cleaned/", f"{temp_dir}{args['dataset_type']}/{args['method']}/log/diffs/")


        print('The result of ', args['dataset_type'])

    def rm_train_music_copy(self):
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/train/cl/
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/test/cl/
   
    def cut_dataset(self, keep_every=10, ram=True, source_path='/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full', destination = '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames', ):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.ls_destination = destination
        self.source_path = source_path
        self.destination = destination
        self.keep_every = keep_every
        self.ram = ram
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        if self.ram:
            if not os.path.exists('/home/smoothjazzuser/Desktop/ram/frames/'):
                os.makedirs('/home/smoothjazzuser/Desktop/ram/frames/')
            self.destination = '/home/smoothjazzuser/Desktop/ram/frames'
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/frames/
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
        else: 
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            #del /frames/ folder
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            #create new /frames/ folder
            !mkdir /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            print('Saving to RAM')
        !mkdir -p /tmp/empty

        self.folders_list = sorted(glob(os.path.join(self.source_path, '*')))
        print(len(self.folders_list))

        # create all desitnation folders, if they don't exist
        for folder in self.folders_list:
            if not os.path.exists(os.path.join(self.destination, folder.split('/')[-1])):
                os.makedirs(os.path.join(self.destination, folder.split('/')[-1]))

        

        c = Parallel(n_jobs=12)(delayed(copy_frames)(folder, self.destination, self.keep_every) for folder in self.folders_list)
        del c
        if ram:
            !ln -s /home/smoothjazzuser/Desktop/ram/frames/ /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/

   
    def analysis(self, video=True, metric_only=False):
        # change working directory in jupyter notebook
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.anomaly_score_total_list = np.load(f"{temp_dir}{self.dataset}/{self.method}/log/anomaly_score_total_list.npy", allow_pickle=True)
        self.feature_distance_list = np.load(f"{temp_dir}{self.dataset}/{self.method}/log/feature_distance_list.npy", allow_pickle=True)
        self.psnr_list = np.load(f"{temp_dir}{self.dataset}/{self.method}/log/psnr_list.npy", allow_pickle=True)
        self.label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{self.dataset}.npy", allow_pickle=True).tolist()[0]
        self.x = self.anomaly_score_total_list
        self.name = self.key = list(np.array(self.psnr_list).reshape(1).tolist()[0].keys())[0]
        print('len x',len(self.x),"len y",len(self.label_list), "key:", self.key)
        ############################################################################################################
        self.psnr = normalize_list(np.array(self.psnr_list).reshape(1).tolist()[0][self.name])
        self.feature_distance = normalize_list(self.feature_distance_list.tolist()[self.name])
        self.adjustment = 4 if self.method == 'pred' else 0
        self.x = self.x[self.adjustment:]
        self.label_list = self.label_list[self.adjustment:]
        self.psnr = self.psnr[self.adjustment:]
        self.feature_distance = self.feature_distance[self.adjustment:]
        self.x_shape = self.x.shape[0]
        self.start = 0
        self.end = int(self.x_shape)
        ############################################################################################################
        if self.metric == 'anomaly_score':
            self.x = self.x[self.start:self.end]
        elif self.metric == 'feature_distance':
            self.x = self.feature_distance[self.start:self.end]
        elif self.metric == 'psnr':
            self.x = self.psnr[self.start:self.end]

        self.label_list = self.label_list[self.start:self.end]
        self.label_points = {'x':[], 'y':[]}
        self.label_plot = {'x':[], 'y':[]}
        for i in range(len(self.label_list)):
            if self.label_list[i] == 1:
                self.label_points['x'].append(i)
                self.label_points['y'].append(self.x[i])
                
                self.label_plot['y'].append(self.x[i])
            else:
                self.label_plot['y'].append(0)
            self.label_plot['x'].append(i)

        stock, start, end, x, y = False, False, False, self.x, self.label_points
        self.data = x
        self.data = self.normalize_data(self.data)
        self.label_points = y
        self.data = pd.Series(self.data)
        self.label_points = pd.DataFrame.from_dict(self.label_points, orient='index')
        self.label_points = self.label_points.transpose() 
        self.zeros = pd.DataFrame(np.zeros(self.data.shape[0]), columns=['zeros'])
        ############################################################################################################
        
        self.TPTP = False
        self.plot(window=CHANGE_WINDOW , alpha=0.5,  plots = ['bet_std_mean', 'std', 'data','mean'])

        
        if not metric_only:
            os.makedirs(f"home/smoothjazzuser/Desktop/output", exist_ok=True)
            self.show_plot(save=True, score_str=self.TPTP)
        else:
            video = False

        
        if video:
            self.r = int((len(self.label_list)-1)/1)
            self.preds = sorted(glob(f"{temp_dir}{self.dataset}/{self.method}/log/preds/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            self.diffs = sorted(glob(f"{temp_dir}{self.dataset}/{self.method}/log/diffs/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            self.ground_truth = []
            self.folders = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2]))
            self.files = [sorted(glob(f"{file}*.jpg") + glob(f"{file}*.png"), key=lambda x: int(x.split('/')[-1].split('.')[0])) for file in self.folders]
            for folder in sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2])):
                for file in sorted(glob(f"{folder}*.jpg") + glob(f"{folder}*.png"), key=lambda x: int(x.split('/')[-1].split('.')[0])):
                    self.ground_truth.append(plt.imread(file))

            self.preds = [plt.imread(pred) for pred in self.preds]
            #self.preds = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plt.imread)(pred) for pred in self.preds)
            self.diffs = [plt.imread(diff) for diff in self.diffs]
            #self.diffs = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plt.imread)(diff) for diff in self.diffs)
            !rm -rf /tmp/*.jpg
            !rm -rf /tmp/*.png
            !rm -rf /tmp/*.svg
            #split r into 10 chunks
            self.rr = list(range(self.r))
            print(f"rr: {len(self.rr)}, preds: {len(self.preds)}, diffs: {len(self.diffs)}, ground_truth: {len(self.ground_truth)}")
            a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[0:int(len(self.rr)/2)])
            del a
            gc.collect()
            a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[int(len(self.rr)/2):-1])
            del a
            gc.collect()
            del self.preds
            del self.diffs
            del self.ground_truth
            gc.collect()
            self.a = imageio.imread(f"/tmp/0.jpg").shape
            self.h,self.w = self.a[0], self.a[1]
            self.frames = Parallel(n_jobs=12, backend='multiprocessing')(delayed(load_frames)(i, self.h, self.w) for i in tqdm(range(self.r-1))) 
            !rm -rf /tmp/*.jpg
            !rm -rf /tmp/*.svg
            !rm -rf /tmp/*.png
            #delete all variables except frames
            gc.collect()
            if not os.path.exists(f"{temp_dir.replace('ram/temp', 'output')}"):
                os.mkdir(f"{temp_dir.replace('ram/temp', 'output')}")
            imageio.mimsave(f"{temp_dir.replace('ram/temp', 'output')}folder-{self.name}_{methodd}_{self.TPTP}.mp4", self.frames, fps=10)
            del self.frames
            gc.collect()
        else:
            !rm -rf /tmp/*.jpg
            !rm -rf /tmp/*.png
            !rm -rf /tmp/*.svg
            gc.collect()
            return self.score
   
    def normalize_data(self, data):
        #return data
        return (data - data.min()) / (data.max() - data.min())
   
    def download_data(self, ticker, start_date, end_date):
        self.data = pdr.get_data_yahoo(ticker, start_date, end_date)
        self.data = self.data.dropna()
        print(self.data.head())
        #normalize between 0 and 1
        self.data = self.normalize_data(self.data['Adj Close'])
        return self.data
   
    def rolling_average(self, data, window):
        roll = data.rolling(window).mean()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return  roll
    
    def rolling_std(self, data, window):
        roll = data.rolling(window).std()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return roll
    
    def gradient_at_each_point(self, data, window=False):
        if window: 
            self.gradient = np.gradient(data.rolling(window))
        else:
            self.gradient = np.gradient(self.data)
        self.gradient = pd.Series(self.gradient)
    
    def points_above_rollingaverage_and_bellow_rollingstd(self, data, window, rolling=False):
        if rolling:
            rolling_mean = self.rolling_average(data, window)
            self.gradient_at_each_point(rolling_mean, window)
            rolling_std = self.rolling_std(data, window)
            self.points = data[(data > rolling_mean) & (data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        else:
            rolling_std = self.rolling_std(data, window)
            self.gradient_at_each_point(data, False)
            self.points = data[(data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        if not self.TPTP:
            self.TPTP, (TP, FP, FN, NUMBER_ANOMALIES, TOTAL_ANOMALIES, ANOMALLY_ASSIGNMENTS) = FP_TP(labels=self.label_list, preds=self.points)
            self.score = FP/FN#/NUMBER_ANOMALIES
            self.TOTAL_ANOMALIES = TOTAL_ANOMALIES
            self.ANOMALLY_ASSIGNMENTS = ANOMALLY_ASSIGNMENTS
            print(self.score)
            return self.score

    def plot(self, window=10, alpha=0.5, plots = ['data', 'mean', 'std', 'exponential', 'troughs', 'bet_std_mean', 'gradient']):
        plt.figure(figsize=(int(len(self.data)/76*1.5), 7))

        if 'mean' in plots:
            rolling_mean = self.rolling_average(self.data, window)
            plt.plot(rolling_mean, label='Rolling Mean')
        if 'std' in plots:
            rolling_std = self.rolling_std(self.data, window)
            plt.plot(rolling_std, label='Rolling Std', color='green')
        if 'exponential' in plots:
            exponential_smoothing = self.exponential_smoothing(self.data, alpha)
            plt.plot(exponential_smoothing, label='Exponential Smoothing')
        if 'troughs' in plots:
            troughs = self.lines_at_troughs(self.data, window)
            plt.plot(troughs, label='Lines at Troughs')
        if 'bet_std_mean' in plots:
            self.points_above_rollingaverage_and_bellow_rollingstd(self.data, window, rolling=False)
            # point y values should equal to data y values
            plt.plot(self.points, 'ro', label='Flagged points', color='purple', markersize=10, alpha=0.5, marker='o')
            # plot these points with y-values = data y values
        #if 'prediction_bins' in plots:
            ## uses the data we calculated in points_above_rollingaverage_and_bellow_rollingstd
            #points_bins = self.TOTAL_ANOMALIES

        if 'data' in plots:
            plt.plot(np.arange(len(self.data)), self.data, color='black', label='data', linewidth=0.6)
            plt.scatter(np.arange(len(self.data)), self.data, color='black', label='data', s=4)
            plt.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        plt.legend(loc='upper left')
        #return all plots
        self.all_plots = plt.gcf()
        #del plot
        plt.close('all')
        return self.all_plots

    def show_plot(self, save=False, score_str=""):
        if save:
            self.all_plots.savefig(f"/home/smoothjazzuser/Desktop/output/folder-{self.name}_{methodd}_{score_str}.svg")
        plt.show()
        return self.all_plots

    def Train (self, c=channels, loss_sections=1, method='blur', method_ = 'recon', dataset_type='bugs', image_size=dims, downscale=True, epochs= 100):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/MNAD/")

        args = {'gpus': None, 'batch_size': 30, 'test_batch_size': 1, 'epochs': epochs, 'loss_compact': 0.15, 'loss_separate': 0.15, 'h': image_size, 'w': image_size, 'c': 3, 'lr': 2e-4, 'method': 'recon', 't_length': 1, 'fdim': 512, 'mdim': 512, 'msize': 10, 'num_workers': 15, 'num_workers_test': 1, 'dataset_type': 'bugs', 'dataset_path': './dataset', 'exp_dir': 'log', 'patience': 3}

        gpus = "0"
        os.environ["CUDA_VISIBLE_DEVICES"]=gpus
        torch.backends.cudnn.enabled = True

        train_folder = args['dataset_path']+"/"+args['dataset_type']+"/training/frames"
        test_folder = args['dataset_path']+"/"+args['dataset_type']+"/testing/frames"

        # Loading dataset
        train_dataset = DataLoader(train_folder, transforms.Compose([transforms.ToTensor(),]), resize_height=args['h'], resize_width=args['w'], time_step=args['t_length']-1)
        test_dataset = DataLoader(test_folder, transforms.Compose([transforms.ToTensor(),]), resize_height=args['h'], resize_width=args['w'], time_step=args['t_length']-1)

        train_size = len(train_dataset)
        test_size = len(test_dataset)

        train_batch = data.DataLoader(train_dataset, batch_size = args['batch_size'], shuffle=True, num_workers=args['num_workers'], drop_last=True)
        test_batch = data.DataLoader(test_dataset, batch_size = args['test_batch_size'], shuffle=False, num_workers=args['num_workers_test'], drop_last=False)
        
        model = convAE(args['c'], memory_size = args['msize'], feature_dim = args['fdim'], key_dim = args['mdim'])
        params_encoder =  list(model.encoder.parameters()) 
        params_decoder = list(model.decoder.parameters())
        params = params_encoder + params_decoder
        optimizer = torch.optim.Adam(params, lr = args['lr'])
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max =args['epochs'])
        model.cuda()

        log_dir = os.path.join('./exp', args['dataset_type'], args['method'], args['exp_dir'])
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)


        if downscale:
            def loss_func_mse(x, y):
                x, y = F.interpolate(x, size=(args['h'], args['w'])), F.interpolate(y, size=(args['h'], args['w'])) #mode = nearest-exact
                #fourier transform
                x = x.squeeze()
                y = y.squeeze()

                loss = F.mse_loss(x,y)
                return loss
        else:
            loss_func_mse = nn.MSELoss(reduction='none')

        # Training

        m_items = F.normalize(torch.rand((args['msize'], args['mdim']), dtype=torch.float), dim=1).cuda() # Initialize the memory items

        e = 0
        for epoch in tqdm(range(args['epochs'])):
            e+=1
            labels_list = []
            model.train()
            
            start = time.time()
            kkk = 0
            #tqdm enumerate
            for j,(imgs) in tqdm(enumerate(train_batch), total=len(train_batch)):
                imgs = Variable(imgs).cuda()
                outputs, _, _, m_items, softmax_score_query, softmax_score_memory, separateness_loss, compactness_loss = model.forward(imgs, m_items, True)
                optimizer.zero_grad()
                loss_pixel = torch.mean(loss_func_mse(outputs, imgs))
                loss = loss_pixel + args['loss_compact'] * compactness_loss + args['loss_separate'] * separateness_loss
                loss.backward(retain_graph=True)
                optimizer.step()
            scheduler.step()

            #early stopping ---> based on self.score
            if not self.model: 
                self.model = model
                self.m_items = m_items
                self.best_score = 100000000000
                self.patience = 0
            else:
                self.swap_test_folders("1001")
                self.Evaluate(metric_only=True) # update the predictions
                self.analysis(metric_only=True, image_size=dims) #update self.score
                os.makedirs('/tmp/empty', exist_ok=True)
                !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/temp/
                pid = os.getpid()
                os.system("kill -9 $(ps -o pid= --ppid %d)" % pid)
                gc.collect()
                if self.score < self.best_score:
                    self.patience = 0
                    self.best_score = self.score
                    self.model = model
                    self.m_items = m_items
                    self.best_epoch = epoch
                else:
                    self.patience += 1
                    if self.patience > args['patience']:
                        print("Early stopping")
                        model = self.model
                        m_items = self.m_items
                        break


            
            print('----------------------------------------')
            print('Epoch:', epoch+1)
            print('Loss: Reconstruction {:.6f}/ Compactness {:.6f}/ Separateness {:.6f}'.format(loss_pixel.item(), compactness_loss.item(), separateness_loss.item()))
            print('Memory_items:')
            print(m_items)
            print('----------------------------------------')
            
        print('Training is finished')
        if not os.path.exists(log_dir+temp_dir):
            os.makedirs(log_dir+temp_dir, exist_ok=True)

        torch.save(model, os.path.join(log_dir+temp_dir, 'model.pth'))
        torch.save(m_items, os.path.join(log_dir+temp_dir, 'keys.pt')) 

        return self.best_score, self.best_epoch

run = anomaly(method='recon', metric='anomaly_score', std_correction='false', dataset="bugs")

In [6]:
if not os.path.exists(temp_dir.replace('temp', 'frames')):
    run.swap_test_folders('all')
    run.cut_dataset(keep_every=1)
    print('cut dataset')

else:
    print('Cut training dataset already exists')

Cut training dataset already exists


In [7]:
run.swap_test_folders('1001')
_, best_epoch = run.Train(epochs=100)


if not os.path.exists('/home/smoothjazzuser/Desktop/output'):
    os.mkdir('/home/smoothjazzuser/Desktop/output/')

run.swap_test_folders('1000')
for loss_type in ['median_blur', 'MSE', 'gaussian_blur']:
    for win in [10, 30, 60, 90]:
        FP_WINDOW  = win
        for change in [5, 10, 20]:
            CHANGE_WINDOW = change
            for kernel in [5, 7, 9]:
                KERNEL_SIZE = kernel

                #load df
                df = pd.read_csv('results.csv')

                methodd = loss_type
                run.Evaluate(image_size=dims)
                run.analysis(video = video, metric_only=True)

                # add data to df
                df.add(run.score, FP_WINDOW, CHANGE_WINDOW, run.FP, run.TP, run.ANOMALIES, "MNAD", best_epoch, loss_type, "MSE", KERNEL_SIZE)

                # remove any duplicates 
                df = df.drop_duplicates(subset=['SCORE','FP_WINDOW', 'CHANGE_WINDOW', 'FP', 'TP', 'ANOMALIES', 'ARCHITECHURE', 'EPOCHS', 'INFERENCE_LOSS_TYPE', 'TRAINING_LOSS_TYPE', "KERNEL_SIZE"], keep='first')

                # del results.csv
                if os.path.exists('results.csv'):
                    os.remove('results.csv')
                    try:
                        df.to_csv('results.csv', index=False)
                    except:
                        df.to_csv(f'results{len(glob("./*.csv") + glob("./*/*.csv"))}.csv', index=False)
                
                
                os.makedirs('/tmp/empty', exist_ok=True)
                !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/temp/
                del run

                #cleanup memory leak from multiprocessing
                # get current procress id
                pid = os.getpid()
                #kill all child processes of pid
                os.system("kill -9 $(ps -o pid= --ppid %d)" % pid)
                gc.collect()
                run = anomaly(method='recon', metric='anomaly_score', std_correction='false', dataset="bugs")

                try:
                    shutil.rmtree('/home/smoothjazzuser/Desktop/ram/', ignore_errors=True)
                    os.mkdir(temp_dir)
                    os.mkdir(temp_dir +'cleaned/')
                except:
                    pass

['1001']
(1, 49272)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/9920 [00:00<?, ?it/s]

In [ ]:
sfbjfjd

In [ ]:

folders_to_test = ['12', '23', '34', '54', '59', '60', '61','109', '117', '130']
folders_to_test = [
    '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'
    ]
#folders_to_test = ['117']
if not os.path.exists('/home/smoothjazzuser/Desktop/output'):
    os.mkdir('/home/smoothjazzuser/Desktop/output/')

for folder in folders_to_test:
    run.swap_test_folders(folder)
    run.Evaluate(image_size=dims)
    run.analysis(video = video)
    
    os.makedirs('/tmp/empty', exist_ok=True)
    !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/temp/
    del run

    #cleanup memory leak from multiprocessing
    # get current procress id
    pid = os.getpid()
    #kill all child processes of pid
    os.system("kill -9 $(ps -o pid= --ppid %d)" % pid)
    gc.collect()
    run = anomaly(method='recon', metric='anomaly_score', std_correction='false', dataset="bugs")

    try:
        shutil.rmtree('/home/smoothjazzuser/Desktop/ram/', ignore_errors=True)
        os.mkdir(temp_dir)
        os.mkdir(temp_dir +'cleaned/')
    except:
        pass

In [ ]:

# delete all files containing "-jvsc-"
!find ./ -name "*-jvsc-*" -type f -delete

In [ ]:
# get current procress id
pid = os.getpid()

#kill all child processes of pid
os.system("kill -9 $(ps -o pid= --ppid %d)" % pid)


In [ ]:
#kill all python
!kill -9 $(ps -o pid= -C python)
